In [1]:
%pylab inline
#Autoreload so changing py files works
%load_ext autoreload
%autoreload 2
from Util import *
from superpoint import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Path to input video (0 to use webcam)
inputpath = 0#'data/chair.mp4'
#Set resize to a tuple to resize frames
resize = False#(800, 450)
#Modulus
skip = 1
#How many frames should be buffered (Larger numbers will cause lag!)
buffer = 3
#Total frames that should be ran
max_frames = 9999
#Show images as they are corrected in a CV frame
show = True
#Enable flipping if images aren't in landscape format
flip = False

#No SuperPoint yet
extractor = SIFTFeatureExtractor()
matcher = BFMatcherHomographyGenerator()
#Read frames
def read():
    ret, frame = cap.read()
    if flip and frame.shape[0] < frame.shape[1]:
        frame = cv2.transpose(frame)
        frame = cv2.flip(frame, 1)
    if resize:
        frame = cv2.resize(frame, resize)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    color = frame#cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return color, gray

frames = []
cap = cv2.VideoCapture(inputpath)
correctedFrames = []
count = 0
H = np.array([[1,0,0],[0,1,0],[0,0,1]])

#Read first frame
color1, gray1 = read()
correctedFrames.append(color1)
kps1, desc1 = extractor.extractFeatures(gray1)
frames.append(gray1)

while cap.isOpened():
    #Read in next frame
    color2, gray2 = read()
    count = count + 1
    #Allow frame skipping
    if count > max_frames: break
    if count % skip == 0:
        #For the first few frames
        if len(frames) < buffer:
            #Add frame to buffer
            frames.append(gray2)
            #Extract features and find Homography
            kps2, desc2 = extractor.extractFeatures(gray2)
            HG = matcher.findHomography(kps2, desc2, kps1, desc1)
            H = H @ HG
            corrected = cv2.warpPerspective(color2, H, (color2.shape[1], color2.shape[0]))
            kps1 = kps2
            desc1 = desc2
        else:
            #Add frame to buffer and push out front of buffer
            del frames[0]
            frames.append(gray2)
            H = np.array([[1,0,0],[0,1,0],[0,0,1]])
            #Extract features and find Homography
            kps1, desc1 = extractor.extractFeatures(frames[0])
            #Multiplies together the Homographies of the frames in the buffer
            for i in range(1, len(frames)):
                kps2, desc2 = extractor.extractFeatures(frames[i])
                HG = matcher.findHomography(kps2, desc2, kps1, desc1)
                H = H @ HG
                kps1 = kps2
                desc1 = desc2
            #Warp current image
            corrected = cv2.warpPerspective(color2, H, (color2.shape[1], color2.shape[0]))
        
        correctedFrames.append(corrected)
        #Render frame to CV window
        if show:
            cv2.imshow('frame', corrected)
    #Press space to end
    if cv2.waitKey(32) == 32:
        break
    
cap.release()